In [1]:
import keras
import time
import numpy as np
from PIL import Image
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D

/home/chawat/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
K.set_learning_phase(1)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.85)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

with tf.device('/cpu:0'):
    X = tf.placeholder(dtype = tf.float32, shape = (1, None, None, 3))
    Y = tf.placeholder(dtype = tf.float32, shape = (None, 5))

In [3]:
vgg = VGGFace(include_top=False, input_tensor = X)
for layer in vgg.layers:
    layer.trainable = False

In [4]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, None, None, 64)    1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, None, None, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, None, None, 64)    0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, None, None, 128)   73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, None, None, 128)   147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, None, None, 128)   0         
__________

In [5]:
p1 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=13).output)
p1 = BatchNormalization()(p1)
p2 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=17).output)
p2 = BatchNormalization()(p2)
p3 = Conv2D(filters=512, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p2))
p3 = BatchNormalization()(p3)
p4 = Conv2D(filters=512, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p3))
p4 = BatchNormalization()(p4)

p1 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(p1)
p1 = BatchNormalization()(p1)
p2 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(p2)
p2 = BatchNormalization()(p2)
p3 = Conv2D(filters=256, kernel_size=5, strides=1, padding='same')(p3)
p3 = BatchNormalization()(p3)
p4 = Conv2D(filters=256, kernel_size=5, strides=1, padding='same')(p3)
p4 = BatchNormalization()(p4)

p1 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(p1)
p1 = BatchNormalization()(p1)
p2 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(p2)
p2 = BatchNormalization()(p2)
p3 = Conv2D(filters=128, kernel_size=5, strides=1, padding='same')(p3)
p3 = BatchNormalization()(p3)
p4 = Conv2D(filters=128, kernel_size=5, strides=1, padding='same')(p4)
p4 = BatchNormalization()(p4)



p11 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p12 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p12 = Activation('relu')(p12)
p21 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p22 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p22 = Activation('relu')(p22)
p31 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p3))
p32 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p3)
p32 = Activation('relu')(p32)
p41 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p4))
p42 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p4)
p42 = Activation('relu')(p42)

p12 = K.concatenate(tensors=[p11, p12], axis = -1)
p22 = K.concatenate(tensors=[p21, p22], axis = -1)
p32 = K.concatenate(tensors=[p31, p32], axis = -1)
p42 = K.concatenate(tensors=[p41, p42], axis = -1)

In [6]:
pred = [K.reshape(p12, (-1, 5)), K.reshape(p22, (-1, 5)), K.reshape(p32, (-1, 5)), K.reshape(p42, (-1, 5))]

In [7]:
pred = K.concatenate(pred, axis = 0)

In [8]:
def NMS(boxes = pred):    
    Z0 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[:, 1:], (-1, 4)), scores=K.reshape(boxes[:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    A = K.reshape(K.tf.gather_nd(boxes, K.reshape(Z0, (-1, 1))), (-1, 5))
    return A

In [9]:
def sigmoid(X):
    return 1/(1 + K.exp(-X))

def relu(X):
    return K.maximum(X, 0)

def loss(Y, y, a=0.8):
    if K.tf.size(Y) != K.tf.size(y):
        n = K.tf.truncatediv(K.tf.size(Y),  K.constant(value = 5, dtype=K.tf.int32))
        y_ = y[:n]
        
    softMax = K.mean(-(Y[:, 0] * K.log(y_[:, 0]) + (1 - Y[:, 0]) * K.log(1 - y_[:, 0])))
    mse = K.mean(K.square(Y[:, 1:] - y_[:, 1:]))
    return 1.25 * softMax + 0.8 * mse + 2 * K.mean(y[n:])

In [10]:
bbox = NMS()

In [11]:
Loss = loss(Y, bbox)

In [12]:
opt = K.tf.train.AdamOptimizer(learning_rate=0.0000009)
train_ = opt.minimize(Loss)

In [13]:
sess.run(K.tf.global_variables_initializer())

In [14]:
def train(epochs = 10):
    for e in range(epochs):
        cost = 0
        _, L = sess.run([train_, Loss], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg"))/255, axis=0), Y: np.random.normal(size = (2, 5)).reshape((-1, 5))})      
        cost += L
        if e % 10 == 0:
            print("epoch {}, iter {} , loss {}".format(e, 0 , cost))

In [16]:
train(epochs = 100)

epoch 0, iter 0 , loss 16.003742218017578
epoch 10, iter 0 , loss 4.756992340087891
epoch 20, iter 0 , loss 2.186253786087036
epoch 30, iter 0 , loss 3.5085835456848145
epoch 40, iter 0 , loss 7.389452934265137
epoch 50, iter 0 , loss 4.697878837585449
epoch 60, iter 0 , loss 4.023492813110352


KeyboardInterrupt: 